Install dependencies

In [21]:
! pip install pandas;
! pip install pgmpy;

Import dependencies

In [22]:
import pandas as pd
import math 
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination

Initialise Data

In [23]:
Quali = "Qualifikation"
S = "Schnitt"
BL = "Bundesland"
M = "Mathe"
P = "Physik"
D = "Deutsch"
ST = "Schultyp"
OLTM = "OLT-Mathe"
OLTD = "OLT-Deutsch"
SFT = "Studierfähigkeitstest"
A = "Alter"
Geschl = "Geschlecht"
JEDE = "Jahreseinkommen der Eltern"
SB = "Staatsbürgerschaft"
SG = "Studiengang"
Absch= "Abschluss"

Load data, specify data types and handle missing values

In [24]:
init_data = pd.read_csv('p001_1.csv', sep=';',dtype={
    Quali: 'string'
})

# Define data types for numerical columns
numerical_columns = [S, M, D, P, OLTM, OLTD, SFT, A, JEDE, Absch]
values = [S, Absch, M, D, P]

#Change a Komma to a dot to convert values 
for column in values:
    init_data[column] = init_data[column].str.replace(",",".")

for column in numerical_columns:
    init_data.loc[:, column] = pd.to_numeric(
        init_data[column], errors="coerce"
    )

# Handle missing values in categorical columns
categorical_columns = [Quali, BL, ST, Geschl, SB, SG]
for column in categorical_columns:
    # Replace missing values with the most frequent category
    init_data.loc[:, column].fillna(init_data[column].mode()[0], inplace=True)
    
display(init_data)

,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,2.7,Baden-Württemberg,2.3,2.2,2.1,Allgemeinbildendes Gymnasium,63,62,685.0,19,m,47000,deutsch,Maschinenbau,3.0
1,Meister,1.6,Nordrhein-Westfalen,NaN,NaN,NaN,n.a.,36,59,NaN,25,m,87000,deutsch,Maschinenbau,NaN
2,Abitur,1.8,Baden-Württemberg,1.0,1.1,1.2,Wirtschaftsgymnasium,96,94,NaN,18,w,115000,deutsch,Soziale Arbeit,1.1
3,Abitur,1.1,Baden-Württemberg,2.5,2.7,1.9,Technisches Gymnasium,70,76,NaN,17,m,115000,deutsch,Elektrotechnik,2.6
4,Abitur,1.4,Bayern,2.0,2.1,1.4,Wirtschaftsgymnasium,65,82,NaN,17,m,90000,deutsch,Elektrotechnik,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,1.4,Baden-Württemberg,2.4,2.5,2.5,Allgemeinbildendes Gymnasium,60,65,NaN,19,w,107000,deutsch,Elektrotechnik,3.1
96,Abitur,2.6,Baden-Württemberg,1.1,1.1,1.2,Technisches Gymnasium,100,94,NaN,20,m,80000,deutsch,Wirtschaftswissenschaften,1.5
97,Abitur,1.6,Baden-Württemberg,2.0,2.2,1.7,Technisches Gymnasium,76,86,NaN,19,m,208000,deutsch,Maschinenbau,2.4
98,Abitur,2.6,Bayern,1.5,1.4,1.1,Allgemeinbildendes Gymnasium,80,86,NaN,18,m,110000,deutsch,Maschinenbau,1.8


Modify Data to be digested more easily

In [25]:
def map_absch(value):
    if math.isnan(value):
        return("abgebrochen")
    elif value < 2:
        return("gut")
    else:
        return("bestehen")
                

def map_age(value):
    if math.isnan(value):
        return(value)
    elif  value < 20:
        return("Teen")
    elif  value < 25:
        return("JungAdult")
    else:
        return("Adult")

def map_grade(value):
    if math.isnan(value):
        return(value)
    elif  value < 2:
        return("Sehr Gut")
    elif  value < 3:
        return("Gut")
    elif  value < 4:
        return("Befriedigend")
    elif  value == 4:
        return("Ausreichend")
    else:
        return("mangelhaft")

def map_olt(value):
    if math.isnan(value):
        return(value)
    elif  value < 50:
        return("Schlecht")
    elif  value < 75:
        return("Gut")
    else:
        return("Sehr Gut")

def map_sft(value):
    if math.isnan(value):
        return(value)
    elif  value < 500:
        return("Schlecht")
    elif  value < 750:
        return("Gut")
    else:
        return("Sehr Gut")

new_absch = []
for value in init_data[Absch]:
    new_absch.append(map_absch(value))
init_data[Absch] = new_absch

new_age = []
for value in init_data[A]:
    new_age.append(map_age(value))
init_data[A] = new_age

grades = [M, D, P, S]

for grade in grades:
    new_grade = []
    for value in init_data[grade]:
        new_grade.append(map_grade(value))
    init_data[grade] = new_grade

new_oltd = []
for value in init_data[OLTD]:
    new_oltd.append(map_olt(value))
init_data[OLTD] = new_oltd

new_oltm = []
for value in init_data[OLTM]:
    new_oltm.append(map_olt(value))
init_data[OLTM] = new_oltm

new_sft = []
for value in init_data[SFT]:
    new_sft.append(map_sft(value))
init_data[SFT] = new_sft

display(init_data)

,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,Gut,Baden-Württemberg,Gut,Gut,Gut,Allgemeinbildendes Gymnasium,Gut,Gut,Gut,Teen,m,47000,deutsch,Maschinenbau,bestehen
1,Meister,Sehr Gut,Nordrhein-Westfalen,NaN,NaN,NaN,n.a.,Schlecht,Gut,NaN,Adult,m,87000,deutsch,Maschinenbau,abgebrochen
2,Abitur,Sehr Gut,Baden-Württemberg,Sehr Gut,Sehr Gut,Sehr Gut,Wirtschaftsgymnasium,Sehr Gut,Sehr Gut,NaN,Teen,w,115000,deutsch,Soziale Arbeit,gut
3,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Sehr Gut,Technisches Gymnasium,Gut,Sehr Gut,NaN,Teen,m,115000,deutsch,Elektrotechnik,bestehen
4,Abitur,Sehr Gut,Bayern,Gut,Gut,Sehr Gut,Wirtschaftsgymnasium,Gut,Sehr Gut,NaN,Teen,m,90000,deutsch,Elektrotechnik,bestehen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Gut,Allgemeinbildendes Gymnasium,Gut,Gut,NaN,Teen,w,107000,deutsch,Elektrotechnik,bestehen
96,Abitur,Gut,Baden-Württemberg,Sehr Gut,Sehr Gut,Sehr Gut,Technisches Gymnasium,Sehr Gut,Sehr Gut,NaN,JungAdult,m,80000,deutsch,Wirtschaftswissenschaften,gut
97,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Sehr Gut,Technisches Gymnasium,Sehr Gut,Sehr Gut,NaN,Teen,m,208000,deutsch,Maschinenbau,bestehen
98,Abitur,Gut,Bayern,Sehr Gut,Sehr Gut,Sehr Gut,Allgemeinbildendes Gymnasium,Sehr Gut,Sehr Gut,NaN,Teen,m,110000,deutsch,Maschinenbau,gut


Create Bayesian Network with Nodes and Edges and init_data

In [26]:
# Create Bayesian Network
G = BayesianNetwork()
G.add_nodes_from(init_data.columns)
display(init_data)

# Add edges to the network
G.add_edges_from([
    (ST, SG),  # Schultyp → Studiengang
    (Quali, S),  # Qualifikation → Schnitt
    (S, M),  # Schnitt → Mathe
    (S, D),  # Schnitt → Deutsch
    (S, P),  # Schnitt → Physik
    (P, OLTM),  # Physik → OLT-Mathe
    (M, OLTM),  # Mathe → OLT-Mathe
    (D, OLTD),  # Deutsch → OLT-Deutsch
    (OLTM, Absch),  # OLT-Mathe → Abschluss
    (OLTD, Absch),  # OLT-Deutsch → Abschluss
    (BL, JEDE),  # Bundesland → Jahreseinkommen der Eltern
    (JEDE, SFT),  # Jahreseinkommen der Eltern → Studierfähigkeitstest
    (JEDE, SG),  # Jahreseinkommen der Eltern → Studiengang
    (SG, Absch),  # Studiengang → Abschluss
    (A, SFT),  # Alter → Studierfähigkeitstest
    (Geschl, SFT),  # Geschlecht → Studierfähigkeitstest
    (SB, SFT)  # Staatsbürgerschaft → Studierfähigkeitstest
])

# Use BayesianEstimator to learn the CPDs
G.fit(data=init_data, estimator=BayesianEstimator)

,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,Gut,Baden-Württemberg,Gut,Gut,Gut,Allgemeinbildendes Gymnasium,Gut,Gut,Gut,Teen,m,47000,deutsch,Maschinenbau,bestehen
1,Meister,Sehr Gut,Nordrhein-Westfalen,NaN,NaN,NaN,n.a.,Schlecht,Gut,NaN,Adult,m,87000,deutsch,Maschinenbau,abgebrochen
2,Abitur,Sehr Gut,Baden-Württemberg,Sehr Gut,Sehr Gut,Sehr Gut,Wirtschaftsgymnasium,Sehr Gut,Sehr Gut,NaN,Teen,w,115000,deutsch,Soziale Arbeit,gut
3,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Sehr Gut,Technisches Gymnasium,Gut,Sehr Gut,NaN,Teen,m,115000,deutsch,Elektrotechnik,bestehen
4,Abitur,Sehr Gut,Bayern,Gut,Gut,Sehr Gut,Wirtschaftsgymnasium,Gut,Sehr Gut,NaN,Teen,m,90000,deutsch,Elektrotechnik,bestehen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Gut,Allgemeinbildendes Gymnasium,Gut,Gut,NaN,Teen,w,107000,deutsch,Elektrotechnik,bestehen
96,Abitur,Gut,Baden-Württemberg,Sehr Gut,Sehr Gut,Sehr Gut,Technisches Gymnasium,Sehr Gut,Sehr Gut,NaN,JungAdult,m,80000,deutsch,Wirtschaftswissenschaften,gut
97,Abitur,Sehr Gut,Baden-Württemberg,Gut,Gut,Sehr Gut,Technisches Gymnasium,Sehr Gut,Sehr Gut,NaN,Teen,m,208000,deutsch,Maschinenbau,bestehen
98,Abitur,Gut,Bayern,Sehr Gut,Sehr Gut,Sehr Gut,Allgemeinbildendes Gymnasium,Sehr Gut,Sehr Gut,NaN,Teen,m,110000,deutsch,Maschinenbau,gut


In [27]:
from pgmpy.factors.discrete.CPD import TabularCPD

#cpd_note = TabularCPD("asd", 1, [[],[],[]],
#                      evidence=[], evidence_card=[],
#                      state_names={'':[],
#                                   '':[]})
#print(cpd_note)
for cpd in G.get_cpds():
    print(cpd)

+--------------------------+---------+
| Qualifikation(Abitur)    | 0.53125 |
+--------------------------+---------+
| Qualifikation(FH Reife)  | 0.15625 |
+--------------------------+---------+
| Qualifikation(Meister)   | 0.15625 |
+--------------------------+---------+
| Qualifikation(Techniker) | 0.15625 |
+--------------------------+---------+
+-----------------------+-----+--------------------------+
| Qualifikation         | ... | Qualifikation(Techniker) |
+-----------------------+-----+--------------------------+
| Schnitt(Befriedigend) | ... | 0.33333333333333337      |
+-----------------------+-----+--------------------------+
| Schnitt(Gut)          | ... | 0.33333333333333337      |
+-----------------------+-----+--------------------------+
| Schnitt(Sehr Gut)     | ... | 0.33333333333333337      |
+-----------------------+-----+--------------------------+
+---------------------------------+----------+
| Bundesland(Baden-Württemberg)   | 0.479167 |
+-----------------------

Make predictions with given input data

In [29]:
# Perform inference with incomplete input
# Example: Predict Abschluss for a student with Qualifikation = "Abitur",
# Schultyp = "Allgemeinbildendes Gymnasium", BL = "Baden-Württemberg",
# and A = 22
evidence = {
    Quali: "Abitur",
    ST: "Wirtschaftsgymnasium",
    BL: "Bayern",
    A: "Teen",
    M: "Sehr Gut",
    SFT: "Sehr Gut"
}

# Use Variable Elimination to make predictions
infer = VariableElimination(G)
q = infer.query(variables=[Absch], evidence=evidence)
print(q)

+------------------------+------------------+
| Abschluss              |   phi(Abschluss) |
+========================+==================+
| Abschluss(abgebrochen) |           0.3082 |
+------------------------+------------------+
| Abschluss(bestehen)    |           0.3835 |
+------------------------+------------------+
| Abschluss(gut)         |           0.3082 |
+------------------------+------------------+
